In [14]:
import cv2
import math
from win32api import GetSystemMetrics
import time
import numpy as np
import os

#NEVER USE NP.ADD always use CV2.BITWISE_OR

In [15]:
imagePath = 'Images'
# Image path
imageName = "sample3.PNG"
imagePath = os.path.join(imagePath,imageName)
#road detection works better with images where the scale is smaller per unit pixel 
#(screen distance = lesser real world distance)
#It also works better with a different style of image

# read image 
img = cv2.imread(imagePath)

mountingPoints = []
quadPoints = []

Steps to do test/implement ray masking
1. Plot out viewing quadrilateral
2. Plot out mounting point
3. Mask out for building contours
4. Find corners of building contours

Taking the nearest corner, find the slope of the line joining the mounting point and this nearest corner. Find the intersection point of this line and the farthest edge of the quadrilateral (What if farthest edge is out of bounds ??)

In [16]:
def getBorderContour_text(img):
#function runs only once
#using new bgr values for the new image in low red.

    img_c = img.copy()
    # Upper and lower color limit customized for snazzy maps
    low_red = (55, 55, 255)
    high_yellow = (242,251,256)

    low_gray = (241,241,241)
    high_gray = (244,243,241)

    # create masks
    red_mask = cv2.inRange(img, low_red, low_red )
    
    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(red_mask, cv2.MORPH_DILATE,kernel)

    blank = np.zeros(img.shape, dtype='uint8')

    masked = cv2.bitwise_and(img,img,mask=combined_mask)

    # findcontours
    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        # cv2.drawContours(img, [c], -1, (255,0,255), thickness=1)
        area = cv2.contourArea(c)
        if(area>200):
            for eps in np.linspace(0.001, 0.01, 10):
                # approximate the contour
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, eps * peri, True)
            
            # draw the approximated contour on the image  
            cv2.drawContours(blank, [approx], -1, (255,255,255), thickness=1)
            # cv2.drawContours(blank, [c], -1, 255, thickness=1)


    # cv2.imshow("image",img)
    # cv2.waitKey(0)

    return blank, masked

In [17]:
def click_MountingnScale_points(event, x, y, flags, params):

    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        mountingPoints.append((x,y))

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +str(y), (x,y), font,1, (255, 0, 0), 2)
        cv2.imshow('img', img)


    # checking for right mouse clicks    
    if event==cv2.EVENT_RBUTTONDOWN:
        quadPoints.append((x,y))

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +str(y),(x,y), font, 1,(0, 0, 255), 2)
        cv2.imshow('img', img)

In [18]:
def getCorners(buildingBorderMask, quadPts, blocksize=3):

    # ~~~~~~~~~~~~~ #
    # for obtaining corners for ray masking
    # ~~~~~~~~~~~~~ #

    # Mask may have 3 layers
    operatedImage = cv2.cvtColor(buildingBorderMask, cv2.COLOR_BGR2GRAY)

    # uncomment if you feel image is noisy (not needed)
    # operatedImage = cv2.fastNlMeansDenoisingColored(operatedImage,None,10,10,7,21) 

    # modify the data type -- setting to 32-bit floating point
    operatedImage = np.float32(operatedImage)

    # apply the cv2.cornerHarris method to detect the corners with appropriate values as input parameters
    # increase the second parameter ~ blocksize to get more of the corner shape out
    dest = cv2.cornerHarris(operatedImage, blocksize, 5, 0.07)  

    # Results are marked through the dilated corners
    dest = cv2.dilate(dest, None)

    # a mask of all corners of the building contours
    cornerMask = np.zeros(buildingBorderMask.shape, dtype='uint8')
    cornerMask[dest > 0.01 * dest.max()]=[255, 255, 255]

    # Make a gray scale mask of quadilateral area
    quadMask = np.zeros(buildingBorderMask.shape[:2], dtype='uint8')
    polyPts = np.array( quadPts ,dtype=np.int32)
    polyPts = polyPts.reshape((-1, 1, 2))
    quadMask = cv2.fillPoly(quadMask, pts=[polyPts],color=255)

    # a mask of all corners in quadilateral only
    cornerMask = cv2.bitwise_and(cornerMask,cornerMask,mask=quadMask)

    return cornerMask


In [19]:
def findClusterCenters(img, excludedCornerPoints = []):

    corner_centers = []

    onePixCornerMask = np.zeros(img.shape[:2], dtype='uint8')
    # blank_copy = blank.copy()

    white_mask = np.zeros(img.shape, dtype = 'uint8')

    if(len(img.shape)==3):
        white_mask = cv2.inRange(img, (255,255,255), (255,255,255))
    else:
        white_mask = cv2.inRange(img, 255, 255)


    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(white_mask, cv2.MORPH_DILATE,kernel)


    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for i in cnts:
        M = cv2.moments(i)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
        # print(f"center - {cx},{cy}")
        if([cx,cy] not in excludedCornerPoints):
            onePixCornerMask[cy,cx] = 255
            # cv2.circle(onePixCornerMask, (cx, cy), 1, (255, 255, 255), -1)
            corner_centers.append([cx,cy])


    return corner_centers, onePixCornerMask


In [20]:
def getCorners_w_exclude(img, cornerPoints=[]):
    
    operatedImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # modify the data type
    # setting to 32-bit floating point
    # operatedImage = cv2.fastNlMeansDenoisingColored(operatedImage,None,10,10,7,21) #uncomment if you feel image is noisy (not needed)
    # operatedImage = np.float32(operatedImage)

    # apply the cv2.cornerHarris method
    # to detect the corners with appropriate
    # values as input parameters
    #~~~~~~~~~~~~~#
    #for obtaining corners for ray masking
    #~~~~~~~~~~~~~#
    dest = cv2.cornerHarris(operatedImage, 2, 5, 0.07)

    # Results are marked through the dilated corners
    dest = cv2.dilate(dest, None)

    # for pts in poly_pts:
    #     dest[pts[0]][pts[1]] = 0
    # Reverting back to the original image,
    # with optimal threshold value
    size = img.shape

    cornerMask = np.zeros(size, dtype='uint8')
    cornerMask[dest > 0.01 * dest.max()]=[255, 255, 255]

    points,cornerMask = findClusterCenters(cornerMask,cornerPoints)

    return cornerMask, points

In [21]:
def drawLine(slope, point):
    #draws line segment infinitely from edges of image

    returnImage = np.zeros(img.shape[:2], dtype = 'uint8')

    y_lim,x_lim = returnImage.shape
    px,py,qx,qz = [-1,-1,-1,-1] #initialize variables with 

    if(slope==-1):
        px = point[0]
        py = y_lim

        qx = point[0]
        qy = 0
    else:
        px = x_lim
        py = slope*x_lim + (point[1] - slope*point[0])

        qx = 0
        qy = (point[1] - slope*point[0])
        
    cv2.line(returnImage, (int(px),int(py)),(int(qx), int(qy)), 255, 1)
    return returnImage  

cv2.imshow("one contour", drawLine(1, (200,200)))
cv2.waitKey(0)
cv2.destroyAllWindows()


##### Step 1: Detect all corners in the image and mask out the view quadrilateral.. to get point only in the interior of the quad\n
##### Step 2: Mask out the view quadrilateral in the original img and then run corner detection.. to get all points interior and on edge
##### Step 3: Loop through every line of every contour inside quad
##### Step 4: For each line AND it with the interior corner points
##### Step 5: Isolate the points and their coordinates that intersect with the contour line
##### Step 6: Get the slope from the mounting point to those interior corners
##### Step 7: Drop a line from the interior corners to edge of image on a quad mask so you dont have to find intersection point with further edge
##### Step 8: Fill the quadrilateral made by the line contour, the lines from interior corners or the view quad side wall and the view quad further wall
##### Step 9: OR all the quadrilateral generated inside the loop on the view quad mask

In [22]:
# read and display image 
img = cv2.imread(imagePath)
cv2.imshow("img",img)

# right click = view quadilateral (4+)
# left click = mounting point (1)
cv2.setMouseCallback('img', click_MountingnScale_points)
# points are to be selected in the order - 
# click 1 = nearest edge left point = quadPoints[0]
# click 2 = nearest edge right point = quadPoints[1]
# click 3 = further edge right point = quadPoints[2]
# click 4 = further edge left point = quadPoints[3]

cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
# building contours on black image and building mask
bldg_brdrs, bldg_mask = getBorderContour_text(img)

# gather quadrilateral points
polyPts = np.array( quadPoints ,dtype=np.int32)
polyPts = polyPts.reshape((-1, 1, 2))

# make viewing quadrilateral mask
quadMask = np.zeros(img.shape[:2], dtype='uint8')
quadMask = cv2.fillPoly(quadMask, pts=[polyPts],color=255)

# get building borders inside viewing quadrilateral
selected_bldg_brdrs = cv2.bitwise_and(bldg_brdrs,bldg_brdrs, mask = quadMask)
selected_bldg_brdrs_gray = cv2.cvtColor(selected_bldg_brdrs, cv2.COLOR_BGR2GRAY) # Gray

# 
selected_edge_list = cv2.findContours(selected_bldg_brdrs_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
selected_edge_list = selected_edge_list[0] if len(selected_edge_list) == 2 else selected_edge_list[1]

# all_selected_corner = cv2.bitwise_and(bldg_brdrs, quadMask)
# all_selected_corner = getCorners(all_selected_corner)

# building corners inside viewing quadrilateral
original_corners = getCorners(bldg_brdrs, quadPoints)
original_corners_g = cv2.cvtColor(original_corners, cv2.COLOR_BGR2GRAY)

# Step 1: Detect all corners in the image and mask out the view quadrilateral.. to get point only in the interior of the quad
_, cornerPix = findClusterCenters(original_corners) #cornerPix is all corner on the inside of the view quadrilateral

# Points displayed on img
# imgCornerPix = np.add(img, original_corners)

cv2.imshow("bldg_brdrs",cornerPix)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
# Step 2: Mask out the view quadrilateral in the original img and then run corner detection.. to get all points interior and on edge
bldg_borders_view_masked = cv2.bitwise_and(bldg_brdrs, bldg_brdrs, mask = quadMask)
all_corners_found = getCorners(bldg_borders_view_masked, quadPoints)
all_corners_found = findClusterCenters(all_corners_found)   #all corners on the interior and on edge

In [25]:
# # use two masks to distinguish points on the side edges vs points on the nearest edge
blank = np.zeros(img.shape[:2], dtype='uint8')
sideEdgeMask_1 = cv2.line(blank, quadPoints[1], quadPoints[2], 255, 1) #plot right edge

blank = np.zeros(img.shape[:2], dtype='uint8')
sideEdgeMask_2 = cv2.line(blank, quadPoints[0], quadPoints[3], 255, 1)   #plot left edge

blank = np.zeros(img.shape[:2], dtype='uint8')
nearestEdgeMask = cv2.line(blank, quadPoints[0], quadPoints[1], 255, 1)   #plot nearest edge

blank = np.zeros(img.shape[:2], dtype='uint8')
farthestEdgeMask = cv2.line(blank, quadPoints[2], quadPoints[3], 255, 1)   #plot farthest edge

# cv2.imshow("farthest edge", farthestEdgeMask)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [26]:
quad_outline = np.zeros(bldg_brdrs.shape[:2], dtype='uint8')
cv2.polylines(quad_outline, [polyPts],True, 255, 1)
mountingPoint = mountingPoints[0]

# Step 3: Loop through every line of every contour inside quad
# for selected_edge_contour in selected_edge_list:

selected_edge_contour = selected_edge_list[0]

#mask related to a single contour, to which individual line blocking masks will be OR'ed to 
totalMask = np.zeros(img.shape[:2], dtype = 'uint8')

oneContourPic = np.zeros(img.shape, dtype='uint8')
cv2.drawContours(oneContourPic, [selected_edge_contour], -1, (255,255,255), thickness=1) #draws only selected contour for the loop
oneContourPic = cv2.cvtColor(oneContourPic, cv2.COLOR_BGR2GRAY)   #convert drawn picture into grayscale

cv2.imshow("one contour", oneContourPic)
cv2.waitKey(0)
cv2.destroyAllWindows()

oneContourPic = cv2.Canny(oneContourPic,50,150,apertureSize=3)
# cv2.imshow("one Contour", oneContourPic)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Apply HoughLinesP method to 
# to directly obtain line end points
lines = cv2.HoughLinesP(
            oneContourPic, # Input edge image
            1, # Distance resolution in pixels
            np.pi/180, # Angle resolution in radians
            threshold=10, # Min number of votes for valid line. Use lesser number of votes for smaller lines
            minLineLength=5, # Min allowed length of line
            maxLineGap=10 # Max allowed gap between line for joining them
            )
for points in lines:

    # Extracted points nested in the list
    x1,y1,x2,y2=points[0]

    # Step 4: For each line AND it with the interior corner points ??

    interiorPointCoords_1 = [y1, x1]
    interiorPointCoords_2 = [y2, x2]

    # Step 5: Isolate the points and their coordinates that intersect with the contour line
    # Step 6: Get the slope from the mounting point to those interior corners
    # Step 7: Drop a line from the interior corners to edge of image on a quad mask so you dont have to find intersection point with further edge
    
    joiningLine_1 = drawLine((mountingPoint[1] - interiorPointCoords_1[1])/(mountingPoint[0] - interiorPointCoords_1[0]), interiorPointCoords_1)
    joiningLine_2 = drawLine((mountingPoint[1] - interiorPointCoords_2[1])/(mountingPoint[0] - interiorPointCoords_2[0]), interiorPointCoords_2)
    
    farthestEdgePoint_1 = cv2.bitwise_and(farthestEdgeMask, joiningLine_1)
    farthestEdgePointCoords_1, _ = findClusterCenters(farthestEdgePoint_1)  #more than one cluster might form - worry !!
    farthestEdgePointCoords_1 = farthestEdgePointCoords_1[0]

    farthestEdgePoint_2 = cv2.bitwise_and(farthestEdgeMask, joiningLine_2)
    farthestEdgePointCoords_2, _ = findClusterCenters(farthestEdgePoint_2)  #more than one cluster might form - worry !!
    farthestEdgePointCoords_2 = farthestEdgePointCoords_2[0]

    cv2.imshow("Farthest Edge Point", farthestEdgePoint_2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


    lineMask = np.zeros(img.shape[:2], dtype='uint8')

    # Step 8: Fill the quadrilateral made by the line contour, the lines from interior corners or the view quad side wall and the view quad further wall
    # There is a concern that the points are listed in not the correct order...
    blockPts = np.array( [interiorPointCoords_1, interiorPointCoords_2, farthestEdgePointCoords_1, farthestEdgePointCoords_2] ,dtype=np.int32)
    blockPts = blockPts.reshape((-1, 1, 2))
    lineMask = cv2.fillPoly(lineMask, pts=[blockPts],color=255)

    # Step 9: OR all the quadrilateral generated inside the loop on the view quad mask
    totalMask = cv2.bitwise_or(totalMask, lineMask)


totalMask = cv2.bitwise_not(totalMask)

